In [160]:
from bokeh.io import (
    output_notebook,
    show,
    )

from bokeh.models import (
    BasicTicker,
    ColorBar,
    ColumnDataSource,
    HoverTool,
    LinearColorMapper,
    LogColorMapper,
    LogTicker
    )

from bokeh.palettes import grey, RdYlBu4

from bokeh.plotting import figure

import numpy as np
import pandas as pd

import json
import os
import sqlite3

report_year = ('2016',)

def db_connect():
    db_path = os.path.join('output', 'sales_new_seed.db')
    conn = sqlite3.connect(db_path)
    return conn



In [142]:
# not used in this report
batch_month_totals = '''
                        SELECT sale_date,
                        SUM(sale_value) AS sale_value
                        FROM sales
                        WHERE strftime('%Y-%m', sale_date) = '2014-01'
                        GROUP BY sale_date;'''
# not used in this report
sum_for_month_by_store = '''
                        SELECT sale_date,
                        sales.store_id,
                        SUM(sale_value) AS sale_value
                        FROM sales
                        INNER JOIN stores ON stores.store_id = sales.store_id
                        WHERE strftime('%Y-%m', sale_date) = '2016-01'
                        GROUP BY sales.store_id;'''

sum_for_month_by_county = '''
                        SELECT stores.county_id AS county_id,
                        counties.county_name AS county_name,
                        SUM(sale_value) as sale_value 
                        FROM sales 
                        INNER JOIN stores ON stores.store_id = sales.store_id
                        INNER JOIN counties ON counties.county_id = stores.county_id
                        WHERE strftime('%Y', sale_date) = ?
                        GROUP BY stores.county_id ORDER BY stores.county_id;'''

annual_volume_of_liquor_sold_by_county = '''
                        SELECT stores.county_id AS county_id,
                        counties.county_name AS county_name,
                        SUM(sale_vol_ml) AS sale_volume
                        FROM sales INNER JOIN stores ON stores.store_id = sales.store_id
                        INNER JOIN counties ON counties.county_id = stores.county_id
                        WHERE strftime('%Y', sale_date) = ?
                        GROUP BY stores.county_id ORDER BY stores.county_id;'''

all_store_lat_long_by_year = '''
                        SELECT DISTINCT stores.store_lat AS lat,
                        stores.store_long AS long
                        FROM sales
                        INNER JOIN stores ON stores.store_id = sales.store_id
                        WHERE strftime('%Y', sale_date) = ?
                        AND stores.store_lat NOT NULL
                        AND stores.store_long NOT NULL
                        AND stores.store_lat >= 40.2
                        AND stores.store_long <= -90.0;'''

# run the queries, return a pandas dataframe
annual_sales_by_county_df = pd.read_sql_query(sum_for_month_by_county, db_connect(), params=report_year)
store_locations_by_year_df = pd.read_sql_query(all_store_lat_long_by_year, db_connect(), params=report_year)
annual_volume_of_liquor_sold_by_county_df = pd.read_sql_query(annual_volume_of_liquor_sold_by_county, db_connect(), params=report_year)

# convert penny values to dollars


# calculate ml per dollar spent
# annual_volume_of_liquor_sold_by_county_df['sale_volume'] = annual_volume_of_liquor_sold_by_county_df['sale_volume'].divide(annual_sales_by_county_df['sale_value'], axis='index')

print('sql queries completed')


sql queries completed


In [143]:
annual_sales_by_county_df.loc[:,'sale_value'] /= 100
annual_volume_of_liquor_sold_by_county_df['sale_volume'] = annual_volume_of_liquor_sold_by_county_df['sale_volume'].divide(annual_sales_by_county_df['sale_value'], axis='index')
annual_volume_of_liquor_sold_by_county_df = annual_volume_of_liquor_sold_by_county_df.rename(index=str, columns={'sale_volume': 'ml_per_dollar'})
print('calculated ml per dollar by dividing sale_volume by sale_value')

calculated ml per dollar by dividing sale_volume by sale_value


In [144]:


print('\n', annual_volume_of_liquor_sold_by_county_df.head())


    county_id county_name  ml_per_dollar
0          1       Adair      66.568309
1          2       Adams      74.069910
2          3   Allamakee      65.833661
3          4   Appanoose      62.730819
4          5     Audubon      83.894919


In [145]:
sales_values = [x for x in annual_sales_by_county_df['sale_value'].tolist()]

sales_county_ids = [int(county) for county in annual_sales_by_county_df['county_id']]

sales_county_names = [county for county in annual_sales_by_county_df['county_name']]

annual_sales_data = {
    'sale_sum': sales_values,
    'county_ids': sales_county_ids,
    'county_names': sales_county_names   
}

annual_sales_by_county_cols = ColumnDataSource(annual_sales_data)

In [146]:
output_notebook()

hover = HoverTool(tooltips=[
    ('county', '@county_names'),
    ('2016 Sales Sum', '@sale_sum')
])

bar_county_sales_title = "Total Liquor Sales by county in {}".format(report_year[0])

bar_county_sales_plot = figure(title=bar_county_sales_title, plot_width=600, plot_height=600, tools=[hover])

bar_county_sales_plot.vbar(x='county_ids', width=0.5, bottom=0, top='sale_sum', source=annual_sales_by_county_cols)

show(bar_county_sales_plot)

Loading BokehJS ...

In [147]:

ml_per_dollar = [x for x in annual_volume_of_liquor_sold_by_county_df['ml_per_dollar'].tolist()]

vol_county_ids = [int(county) for county in annual_volume_of_liquor_sold_by_county_df['county_id']]

vol_county_names = [county for county in annual_volume_of_liquor_sold_by_county_df['county_name']]

vol_data = {
    'ml_per_dollar': ml_per_dollar,
    'county_ids': vol_county_ids,
    'county_names': vol_county_names   
}

annual_volume_of_liquor_sold_by_county = ColumnDataSource(vol_data)


In [148]:
output_notebook()

vol_hover_text = '{} ml per dollar spent'.format(report_year[0])

hover = HoverTool(tooltips=[
    ('county', '@county_names'),
    (vol_hover_text, '@ml_per_dollar')
])

bar_volume_by_county_title = "Milliliters purchased per Dollar spent {}".format(report_year[0])

bar_volume_by_county_plot = figure(title=bar_volume_by_county_title, plot_width=600, plot_height=600, tools=[hover])

bar_volume_by_county_plot.vbar(x='county_ids', width=0.5, bottom=0, top='ml_per_dollar', source=annual_volume_of_liquor_sold_by_county)

show(bar_volume_by_county_plot)

Loading BokehJS ...

In [149]:
'''these functions prep or align geo data and match data from a database query 
    with the correct geo data using county names'''

def counties_by_state_number(raw_data):
    counties = []
    for state in raw_data:
        # as soon as the Iowa feauture is found, nuke all the other states
        if state['properties']['STATE'] == '19':
            counties.append(state)
    return counties

def extract_single_county_patch(raw_geojson):
        
    manual_x = []
    manual_y = []
    
    for coord in raw_geojson[0]:
        manual_x.append(coord[0])
        manual_y.append(coord[1])
    
    return (manual_x, manual_y)

def manually_build_patches(raw_geojson):
    manual_xs = []
    manual_ys = []
    names = []
    
    for feature in raw_geojson['features']:
        names.append(feature['properties']['NAME'])
        manual_x, manual_y = extract_single_county_patch(feature['geometry']['coordinates'])
        manual_xs.append(manual_x)
        manual_ys.append(manual_y)
        names.append
    
    return (manual_xs, manual_ys, names)

def build_sale_val_array(df, target_map_data, data_name):
    
    county_df = df.set_index('county_name')
    max_value = df[data_name].max()
    arranged_sale_values = []
    
    for county in target_map_data:
        
        county_sale_value = county_df.loc[county][1]
        arranged_sale_values.append(county_sale_value)
    
    return arranged_sale_values



def build_county_map(geo_data_source, data_to_align, mapped_data_name):
    with open(geo_data_source, 'r', encoding='latin-1') as raw_state_json:
        # returns python object from the raw json file data
        raw_geo_data = json.loads(raw_state_json.read())

    # replace raw geo data with only Iowa counties
    raw_geo_data['features'] = counties_by_state_number(raw_geo_data['features'])

    # build the lists of data in the correct orders for our ColumnData
    manual_x, manual_y, county_names = manually_build_patches(raw_geo_data)

    # biulds a sale_value array aligned in the correct order to match the geo data counties
    mapped_data = build_sale_val_array(data_to_align, county_names, mapped_data_name)

    # source dictionary to pass to ColumnDataSource 
    manual_data_for_map = {
        'xs': manual_x,
        'ys': manual_y,
        'county_name': county_names,
        mapped_data_name: mapped_data
    }
    
    return manual_data_for_map

# --- GLOBAL MAPPING VARIABLES ---
# geojson for counties source: http://eric.clst.org/tech/usgeojson/
geo_data_path = os.path.join('input', 'geojson', 'counties_high_res_gz_2010_us_050_00_500k.json')

store_lats = [x for x in store_locations_by_year_df['lat']]
store_longs = [y for y in store_locations_by_year_df['long']]

store_locations = {
    'x': store_longs,
    'y': store_lats
}

# --- END OF GLOBAL MAPING VARIABLES ---

In [150]:
def display_county_map(col_data_source, field_name, hover_data_label, map_title, **kwargs):
    '''county mapping function,
    passed a column data source,
    a df with indexed countie ID, Name, and Data to be mapped
    currently uses global store location variables'''
    
    hover_tool_def_string = '@' + field_name
    
    hover = HoverTool(tooltips=[
        ("County", "@county_name"),
        (hover_data_label, hover_tool_def_string),
        ], names=['counties'])

    map_palette = grey(16)
        
    print(kwargs['invert'])
    if kwargs['invert']:
        map_palette.reverse()
    
    color_mapper = LinearColorMapper(palette=map_palette,
                                     low=min(col_data_source.data[field_name]),
                                     high=max(col_data_source.data[field_name]))

    iowa_counties_map_title = map_title + ' {}'.format(report_year[0])

    iowa_counties_map = figure(title=iowa_counties_map_title,
                               tools=[hover, 'wheel_zoom', 'pan', 'reset'], x_axis_location=None,
                               y_axis_location=None,
                               width=770,
                               height=500
                              )

    iowa_counties_map.grid.grid_line_color = None

    iowa_counties_map.patches('xs', 'ys', 
                              fill_color={
                                  'field': field_name,
                                  'transform': color_mapper
                              },
                              fill_alpha=0.7, line_color='grey',
                              line_width=0.5,
                              source=col_data_source,
                              name='counties'
                             )

    # store location mapping
    store_location_col = ColumnDataSource(store_locations)

    iowa_counties_map.cross('x', 'y', size=2, color='red', source=store_location_col, name='stores')

    map_color_bar = ColorBar(color_mapper=color_mapper,
                         ticker=BasicTicker(),
                         label_standoff=12,
                         border_line_color=None,
                         location=(0,0),
                         orientation='horizontal')

    iowa_counties_map.add_layout(map_color_bar, 'below')
    
    output_notebook()
    show(iowa_counties_map)


In [151]:
manual_col_map_data = ColumnDataSource(build_county_map(geo_data_path, annual_sales_by_county_df, 'sale_value'))

display_county_map(manual_col_map_data, 'sale_value', 'Annual Total Liqour Sales', 'Liquor Store Locations and Total Annual Sales per County', invert=False)

False


Loading BokehJS ...

In [152]:
ml_per_doll_map_data = ColumnDataSource(build_county_map(geo_data_path, annual_volume_of_liquor_sold_by_county_df, 'ml_per_dollar'))

display_county_map(ml_per_doll_map_data, 'ml_per_dollar', 'ml / dollar', 'Liquor Store Locations and Annual Milliliters per Dollar Spent', invert=True)

True


Loading BokehJS ...

In [153]:
# parse population json 
# source: https://data.iowa.gov/widgets/qtnr-zsrc

county_pop_json_path = os.path.join('input', 'population-data', 'iowa_county_population_per_year.json')

def parse_population_json(pop_json_path):
    with open(pop_json_path, 'r', encoding='latin-1') as raw_county_pop_json:
    # returns python object from the raw json file data
        raw_population_data = json.loads(raw_county_pop_json.read())
        
    county_2016_population = []
    
    labels = ['county_name', 'population']
    
    for row in raw_population_data['data']:
        if row[10][:4] == '2016':
            county_name_pop_snippet = []
            # removes 'county' from the end of each county name without breaking two-word county names
            county_name_pop_snippet.append(' '.join(row[9].split(' ')[:-1]))
            county_name_pop_snippet.append(row[11])
            county_2016_population.append(county_name_pop_snippet)
 
    county_population_df = pd.DataFrame.from_records(county_2016_population, columns=labels)
    return county_population_df
    
county_population_df = parse_population_json(county_pop_json_path)

county_population_df.head()


,county_name,population
0,Adair,7092
1,Adams,3693
2,Allamakee,13884
3,Appanoose,12462
4,Audubon,5678


In [162]:
# annual_volume_of_liquor_sold_by_county_df
# annual_sales_by_county_df

scatter_compare = figure(title = 'Annual Sales vs. ml Purchased per Dollar (Color-coded by Population)')
scatter_compare.xaxis.axis_label = 'Annual Sales'
scatter_compare.yaxis.axis_label = 'ml Purchased per Dollar'

scatter_color_map = LogColorMapper(palette=RdYlBu4, low=3693, high=474045)

scatter_pop_data = {
    'county_name': [x for x in county_population_df['county_name']],
    'county_population': [x for x in county_population_df['population']],
    'sale_value': [x for x in annual_sales_by_county_df['sale_value']],
    'ml_per_dollar': [x for x in annual_volume_of_liquor_sold_by_county_df['ml_per_dollar']]
}

scatter_pop_col_data = ColumnDataSource(scatter_pop_data)

scatter_compare.circle('sale_value',
                       'ml_per_dollar',
                       size=5,
                       fill_color={
                           'field': 'county_population',
                           'transform': scatter_color_map},
                       fill_alpha=0.5,
                       line_color={
                           'field': 'county_population',
                           'transform': scatter_color_map},
#                        legend=field('county_population'),
                       source=scatter_pop_col_data)

color_bar = ColorBar(color_mapper=scatter_color_map,
                     ticker=LogTicker(),
                     label_standoff=12,
                    border_line_color=None,
                    location=(0,0),
                    orientation='horizontal')

scatter_compare.add_layout(color_bar, 'above')

output_notebook()

show(scatter_compare)

Loading BokehJS ...